# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

## Лабораторная работа №4

### JSON

In [1]:
import json

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [4]:
with open('data/contributors_sample.json', 'r') as f:
    contributors = json.load(f)
    
for user in contributors[:3]:
    print(user)
    print()

{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}

{'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970}

{'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}



1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [6]:
domains = set()
for user in contributors:
    email = user['mail']
    if email:
        domain = email.split('@')[-1]
        domains.add(domain)

for domain in domains:
    print(domain)

hotmail.com
yahoo.com
gmail.com


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [11]:
def find_user(username):
    for user in contributors:
        if user['username'] == username:
            return user
    raise ValueError(f"Contributor with username {username} not found.")

user = find_user('uhebert')
print(user)

{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}


In [12]:
user = find_user('uhebert2')

ValueError: Contributor with username uhebert2 not found.

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [14]:
male_count = 0
female_count = 0

for user in contributors:
    gender = user['sex']
    if gender == 'M':
        male_count += 1
    elif gender == 'F':
        female_count += 1

print(f"Количество мужчин: {male_count}")
print(f"Количество женщин: {female_count}")


Количество мужчин: 2064
Количество женщин: 2136


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [16]:
import pandas as pd

contributors_df = pd.DataFrame(contributors, columns=['id', 'username', 'sex'])
print(contributors_df.head())


        id     username sex
0    35193      uhebert   F
1    91970  vickitaylor   F
2  1848091  sheilaadams   F
3    50969     nicole82   F
4   676820       jean67   M


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [23]:
# Загружаем данные из файла recipes_sample.csv в DataFrame
recipes = pd.read_csv('data/recipes_sample.csv')

# Объединяем таблицы recipes и contributors
merged_df = pd.merge(recipes, contributors_df, how='left', left_on='contributor_id', right_on='id')

# Считаем количество людей, информация о которых отсутствует в JSON-файле
missing_contributors = merged_df['username'].isnull().sum()

# Выводим результаты
print(f"Количество людей, информация о которых отсутствует в JSON-файле: {missing_contributors}")
print(merged_df.head())

Количество людей, информация о которых отсутствует в JSON-файле: 15059
                                       name   id_x  minutes  contributor_id  \
0     george s at the cove  black bean soup  44123       90           35193   
1        healthy for them  yogurt popsicles  67664       10           91970   
2              i can t believe it s spinach  38798       30            1533   
3                      italian  gut busters  35173       45           22724   
4  love is in the air  beef fondue   sauces  84797       25            4470   

    submitted  n_steps                                        description  \
0  2002-10-25      NaN  an original recipe created by chef scott meska...   
1  2003-07-26      NaN  my children and their friends ask for my homem...   
2  2002-08-29      NaN            these were so go, it surprised even me.   
3  2002-07-27      NaN  my sister-in-law made these for us at a family...   
4  2004-02-23      4.0  i think a fondue is a very romantic casual di

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [30]:
import pickle

positions = {}

for user in contributors:
    for position in user['jobs']:
        if position not in positions:
            positions[position] = []
        positions[position].append(user['username'])

In [31]:
for job, usernames in positions.items():
    print(job, usernames[:])

Energy engineer ['uhebert', 'annmoore', 'garysilva', 'martinezashley', 'sextonsheila', 'pjames', 'smithjonathan', 'wardjames', 'cwheeler', 'ucarlson', 'robert71', 'johnsontheresa', 'amanda41', 'stacey47', 'timothynelson', 'timothynelson', 'rogersmichael', 'melissa94', 'wmcdaniel', 'charles74', 'smithjennifer', 'clintonjones']
Engineer, site ['uhebert', 'nancy12', 'andrea03', 'catherineross', 'wesley32', 'natalieross', 'rossdoris', 'christophersmith', 'dbooker', 'ericarobertson', 'trantricia', 'tpugh', 'jasonvelez', 'samantha36', 'brandidaniels', 'tenglish', 'reyesbrett', 'austin18', 'vjohnson', 'zmejia', 'daniel04', 'cynthia20', 'morgan15', 'avaldez', 'jessica92', 'laurieholloway', 'baileyvictoria']
Environmental health practitioner ['uhebert', 'jonathanchristian', 'xjohnson', 'dsmith', 'james01', 'nancytaylor', 'ztaylor', 'andrewwoods', 'susan54', 'fmaldonado', 'james74', 'bakerjacob', 'stephanie81', 'whitejoseph', 'qolson', 'hknox', 'gonzalesdaniel', 'tranronald', 'jessegreen', 'step

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [32]:
with open('job_people.pickle', 'wb') as f:
    pickle.dump(positions, f)

with open('job_people.json', 'w') as f:
    json.dump(positions, f, indent=4)

In [33]:
import os

print(f'pickle file size: {os.path.getsize("job_people.pickle")} bytes')
print(f'json file size: {os.path.getsize("job_people.json")} bytes')

pickle file size: 132272 bytes
json file size: 389655 bytes


2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [34]:
with open('job_people.pickle', 'rb') as f:
    job_people = pickle.load(f)

print(job_people)

{'Energy engineer': ['uhebert', 'annmoore', 'garysilva', 'martinezashley', 'sextonsheila', 'pjames', 'smithjonathan', 'wardjames', 'cwheeler', 'ucarlson', 'robert71', 'johnsontheresa', 'amanda41', 'stacey47', 'timothynelson', 'timothynelson', 'rogersmichael', 'melissa94', 'wmcdaniel', 'charles74', 'smithjennifer', 'clintonjones'], 'Engineer, site': ['uhebert', 'nancy12', 'andrea03', 'catherineross', 'wesley32', 'natalieross', 'rossdoris', 'christophersmith', 'dbooker', 'ericarobertson', 'trantricia', 'tpugh', 'jasonvelez', 'samantha36', 'brandidaniels', 'tenglish', 'reyesbrett', 'austin18', 'vjohnson', 'zmejia', 'daniel04', 'cynthia20', 'morgan15', 'avaldez', 'jessica92', 'laurieholloway', 'baileyvictoria'], 'Environmental health practitioner': ['uhebert', 'jonathanchristian', 'xjohnson', 'dsmith', 'james01', 'nancytaylor', 'ztaylor', 'andrewwoods', 'susan54', 'fmaldonado', 'james74', 'bakerjacob', 'stephanie81', 'whitejoseph', 'qolson', 'hknox', 'gonzalesdaniel', 'tranronald', 'jesseg

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [60]:
import xml.etree.ElementTree as ET

# открываем файл и парсим его с помощью ElementTree
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

# инициализируем пустой словарь для хранения шагов
steps_dict = {}

# проходим по каждому рецепту и извлекаем его шаги
for recipe in root.findall('recipe'):
    recipe_id = recipe.find('id').text
    steps = []
    for step in recipe.find('steps').findall('step'):
        steps.append(step.text)
    steps_dict[recipe_id] = steps

# сохраняем словарь в файл steps_sample.json
with open('steps_sample.json', 'w') as f:
    json.dump(steps_dict, f)


In [61]:
with open('steps_sample.json') as f:
    data = json.load(f)

# вывести первые три элемента
for recipe_id, steps in list(data.items())[:3]:
    print(f"Recipe {recipe_id}: {steps}")

Recipe 44123: ['in 1 / 4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes', 'add thyme , oregano and basil', 'saute 5 minutes more', 'add wine and deglaze pan', 'add hot chicken stock and reduce by one-third', 'add worcestershire sauce , tabasco , smoked chicken , beans and broccoli florets', 'simmer 5 minutes', 'add cream , simmer 5 minutes more and season to taste', 'drop in remaining butter , piece by piece , stirring until melted and serve immediately', 'smoked chicken: on a covered grill , slightly smoke boneless chicken , cooking to medium rare', 'chef meskan uses applewood chips and does not allow the grill to become too hot']
Recipe 67664: ['mix all the ingredients using a blender', 'pour into popsicle molds', 'freeze and enjoy !']
Recipe 38798: ['combine all ingredients in a large bowl and mix well', 'shape into one-inch balls', 'cover and refrigerate or freeze until ready to bake', 'preheat oven to 350 degrees', 'place on ungreased baking sheet and

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [69]:
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

recipes_steps = {}
for recipe in root.findall('recipe'):
    recipe_id = recipe.find('id').text
    steps_num = len(recipe.findall('steps/step'))
    if steps_num not in recipes_steps:
        recipes_steps[steps_num] = [recipe_id]
    else:
        recipes_steps[steps_num].append(recipe_id)

for step, id in recipes_steps.items():
    print(step, id[:5])

11 ['44123', '302399', '375376', '140610', '374703']
3 ['67664', '118843', '147477', '367987', '216068']
5 ['38798', '69190', '125195', '306590', '77380']
7 ['35173', '109818', '66932', '223349', '63346']
4 ['84797', '95926', '134085', '192542', '100837']
6 ['44045', '306168', '325714', '276594', '289671']
8 ['107229', '463219', '39172', '371549', '152534']
12 ['453467', '226001', '129919', '121107', '40228']
15 ['50662', '180817', '133326', '82136', '382633']
10 ['503475', '149593', '95534', '60938', '383120']
18 ['200148', '137701', '122791', '213132', '336314']
38 ['310570', '392181', '51546', '56109', '336218']
13 ['141939', '303944', '299968', '217296', '213395']
14 ['250883', '120297', '296983', '166089', '299989']
9 ['342620', '342619', '287778', '376813', '147563']
33 ['129581', '111422', '178749', '513419', '350037']
2 ['116741', '44050', '292568', '327979', '26420']
30 ['487173', '319600', '479954', '524062', '198880']
24 ['447429', '170081', '490199', '403981', '332386']
28 

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [72]:
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

recipes_with_time = []

for recipe in root.findall('recipe'):
    recipe_id = recipe.find('id').text
    steps = recipe.findall('steps/step')
    for step in steps:
        if 'has_minutes' in step.attrib or 'has_hours' in step.attrib:
            recipes_with_time.append(recipe_id)
            break

print(recipes_with_time)

['44123', '35173', '453467', '306168', '50662', '118843', '149593', '200148', '310570', '95534', '109818', '66932', '226001', '141939', '250883', '120297', '223349', '60938', '302399', '342620', '296983', '129581', '325714', '487173', '447429', '137701', '63346', '342619', '383120', '463219', '39172', '216068', '173730', '287778', '437637', '123115', '371549', '376813', '390230', '401605', '306590', '299968', '192542', '147563', '193719', '38852', '250232', '437219', '77380', '21357', '198343', '129919', '375376', '63131', '24760', '375362', '217296', '435816', '198073', '202949', '367828', '140610', '392181', '468143', '176996', '290187', '459738', '111198', '33246', '302498', '165438', '267159', '401283', '428566', '533190', '478546', '40228', '255985', '402246', '180817', '133326', '513963', '213395', '482111', '292147', '235003', '449768', '257696', '53353', '115160', '140172', '392598', '356163', '140173', '289025', '291409', '282812', '489337', '250900', '160494', '245950', '3885

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [74]:
recipes = pd.read_csv('data/recipes_sample.csv')

recipes[recipes['n_steps'].isna()]

tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

steps_dict = {}

for recipe in root.findall('recipe'):
    recipe_id = recipe.find('id').text
    steps = recipe.find('steps')
    n_steps = len(steps.findall('step'))
    steps_dict[recipe_id] = n_steps

for index, row in recipes.iterrows():
    recipe_id = str(row['id'])
    n_steps = row['n_steps']
    if pd.isnull(n_steps):
        if recipe_id in steps_dict:
            recipes.at[index, 'n_steps'] = steps_dict[recipe_id]

print(recipes)

                                               name      id  minutes  \
0             george s at the cove  black bean soup   44123       90   
1                healthy for them  yogurt popsicles   67664       10   
2                      i can t believe it s spinach   38798       30   
3                              italian  gut busters   35173       45   
4          love is in the air  beef fondue   sauces   84797       25   
...                                             ...     ...      ...   
29995  zurie s holey rustic olive and cheddar bread  267661       80   
29996          zwetschgenkuchen  bavarian plum cake  386977      240   
29997   zwiebelkuchen   southwest german onion cake  103312       75   
29998                                   zydeco soup  486161       60   
29999        cookies by design   cookies on a stick  298512       29   

       contributor_id   submitted  n_steps  \
0               35193  2002-10-25     11.0   
1               91970  2003-07-26      3.0 

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [75]:
# проверка наличия пропусков в столбце n_steps
if recipes['n_steps'].isnull().any():
    pass

# преобразование столбца n_steps к целочисленному типу
recipes['n_steps'] = recipes['n_steps'].astype(int)

# сохранение результата в файл
recipes.to_csv('recipes_sample_with_filled_nsteps.csv', index=False)

print(recipes)

                                               name      id  minutes  \
0             george s at the cove  black bean soup   44123       90   
1                healthy for them  yogurt popsicles   67664       10   
2                      i can t believe it s spinach   38798       30   
3                              italian  gut busters   35173       45   
4          love is in the air  beef fondue   sauces   84797       25   
...                                             ...     ...      ...   
29995  zurie s holey rustic olive and cheddar bread  267661       80   
29996          zwetschgenkuchen  bavarian plum cake  386977      240   
29997   zwiebelkuchen   southwest german onion cake  103312       75   
29998                                   zydeco soup  486161       60   
29999        cookies by design   cookies on a stick  298512       29   

       contributor_id   submitted  n_steps  \
0               35193  2002-10-25       11   
1               91970  2003-07-26        3 